### End-to-End with Assembly.ai ###

### Input to Email ###

In [1]:
# !pip install wordcloud
# !pip install pywaffle

In [2]:
#Data Manipulation
import pandas as pd
import datetime as dt
import numpy as np
# from rev_ai import *
# get_transcript()
#Visualization
import matplotlib.pyplot as plt
from chord import Chord
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
from pywaffle import Waffle

#Text manipulation
from gensim.summarization.summarizer import summarize
from gensim.summarization import keywords
from gensim.parsing.preprocessing import remove_stopwords
from rake_nltk import Rake
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm

In [3]:
import sys
import time
import requests

filename = "TED_03.mp4"
 
def read_file(filename, chunk_size=5242880):
    with open(filename, 'rb') as _file:
        while True:
            data = _file.read(chunk_size)
            if not data:
                break
            yield data

headers = {'authorization': "681814e18ef446d8a29a9c4fec447231"}
response = requests.post('https://api.assemblyai.com/v2/upload',
                         headers=headers,
                         data=read_file(filename))

#this is needed for the next call below - 'audio url'
url = response.json()['upload_url']

#print(response.json())

In [9]:
#STEP 2
#THIS CAN ALSO TAKE ONLINE HOSTED FILE WHERE URL CALL IS LOCATED

import requests
endpoint = "https://api.assemblyai.com/v2/transcript"

json = {
  "audio_url": f"{url}", 
  'auto_highlights': True,
  'speaker_labels': True
}

headers = {
    "authorization": "681814e18ef446d8a29a9c4fec447231",
    "content-type": "application/json"
}

response = requests.post(endpoint, json=json, headers=headers)
id = response.json()['id']
response.json()

{'id': 'lx6eehlr4-af12-48c5-ba76-a221a3b6a20d',
 'text': None,
 'words': None,
 'utterances': None,
 'confidence': None,
 'status': 'queued',
 'audio_duration': None,
 'audio_url': 'https://cdn.assemblyai.com/upload/999263d1-de8d-4799-8bbd-20939ac0d014',
 'language_model': 'assemblyai_default',
 'acoustic_model': 'assemblyai_default',
 'punctuate': True,
 'format_text': True,
 'dual_channel': None,
 'webhook_url': None,
 'webhook_status_code': None,
 'speed_boost': False,
 'audio_start_from': None,
 'audio_end_at': None,
 'word_boost': [],
 'boost_param': None,
 'redact_pii': False,
 'auto_highlights': True}

In [46]:
response.json()['status']

'completed'

In [13]:
#STEP 3 - TO GET THE TRANSCRIPT
# import requests

endpoint = f"https://api.assemblyai.com/v2/transcript/{id}"

headers = {
    "authorization": "681814e18ef446d8a29a9c4fec447231",

}

response = requests.get(endpoint, headers=headers)
status = response.json()['status']
while status == 'queued' or status == 'processing':
    response = requests.get(endpoint, headers=headers)
    status = response.json()['status']

transcript_json = response.json()

In [35]:
#Prints out the who json file
# transcript_json

In [98]:
df = pd.DataFrame.from_dict(transcript_json['utterances'])
# df = df.rename(columns = {'start':'Time', 'speaker': 'Speaker', "text": 'Text'})
df
# utterances_df.to_csv('utterances.csv')

,confidence,end,text,start,speaker,words
0,0.957692,12010,"Ryan, let me go to you. Is there enough accoun...",0,A,"[{'text': 'Ryan,', 'confidence': 0.97, 'speake..."
1,0.917333,35390,"A national standpoint. I mean, if you look at ...",12100,B,"[{'text': 'A', 'confidence': 1.0, 'speaker': '..."
2,0.964000,42860,Does justice look like to you in these situati...,35400,A,"[{'text': 'Does', 'confidence': 0.94, 'speaker..."
3,0.990000,44360,And.,43360,B,"[{'text': 'And.', 'confidence': 0.99, 'speaker..."
4,0.939941,109440,Equal punishment. The issue at hand I feel lik...,45850,A,"[{'text': 'Equal', 'confidence': 0.99, 'speake..."


In [106]:
df['num_words_spoken'] = [len(i) for i in df['text']]

In [105]:
Chord.user = "jasonrodneybrooks@gmail.com"
Chord.key = "CP-79642c07-fcad-4b3c-81b3-d3d6bf4be0a6"

#Get chord diagram
links = df.loc[:, ('speaker', 'num_words_spoken')]
links['target'] = links['speaker'].shift(periods=-1)
links['num_words_received'] = links['num_words_spoken'].shift(periods=-1)

names_label = pd.crosstab(links['speaker'], links['target'])
names = names_label.index.to_list()
matrix = names_label.values.tolist()

chord = Chord(matrix, names, arc_numbers = True, 
              conjunction = "&", verb = "spoke to each other ", noun = 'times', 
              colors="d3.schemeSet1",opacity=0.8, symmetric = False,
        padding=0.001,
        width=900,
        label_color="#454545",
        wrap_labels=True,
        margin=0,
        credit=True,
        font_size="20px",
        font_size_large="18px")

chord.show()
# chord.to_html('NAME.html')